In [7]:
from glob import glob
from tqdm import tqdm
import json
import re

In [36]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [8]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas'
]
rejected_words = set(rejected_words)

def check_indon(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    return len(set(string.split()) & rejected_words) > 0

In [15]:
files = sorted(glob('evol-instructs/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

110635

In [16]:
with open(files[-2]) as fopen:
    data = json.load(fopen)
data

{'instruction': 'Sementara anda meneruskan penyelidikan mendalam topik ini, pertimbangkan untuk membangunkan siri pertanyaan yang berkait antara satu sama lain, akhirnya membawa kepada pemahaman menyeluruh tentang konsep yang ada. Setiap pertanyaan perlu memerlukan tahap pemikiran yang lebih tinggi dan pemahaman tentang maklumat yang disajikan, mencabar anda untuk menjelajahi dengan lebih mendalam tentang kompleksiti subjek yang dibincangkan. Matlamat utama adalah mencapai tahap pemahaman yang membolehkan anda untuk mengaplikasikan pengetahuan ke dalam situasi dunia nyata dan membuat keputusan yang informatif berdasarkan maklumat yang diberikan.',
 'output': 'Tentu, saya faham. Anda mencadangkan untuk membina siri pertanyaan yang berkait antara satu sama lain untuk memahami konsep secara menyeluruh. Setiap pertanyaan perlu menggalakkan pemikiran tinggi dan pemahaman yang mendalam tentang maklumat yang disajikan, supaya saya dapat menjelajahi subjek dengan lebih mendalam. Matlamat utama

In [17]:
print(data['instruction'])

Sementara anda meneruskan penyelidikan mendalam topik ini, pertimbangkan untuk membangunkan siri pertanyaan yang berkait antara satu sama lain, akhirnya membawa kepada pemahaman menyeluruh tentang konsep yang ada. Setiap pertanyaan perlu memerlukan tahap pemikiran yang lebih tinggi dan pemahaman tentang maklumat yang disajikan, mencabar anda untuk menjelajahi dengan lebih mendalam tentang kompleksiti subjek yang dibincangkan. Matlamat utama adalah mencapai tahap pemahaman yang membolehkan anda untuk mengaplikasikan pengetahuan ke dalam situasi dunia nyata dan membuat keputusan yang informatif berdasarkan maklumat yang diberikan.


In [26]:
rejected_instructions = [
    'tidak dapat melakukan penulisan semula', 
    'maaf', 
    'not able to fulfill your request',
    'saya tidak dapat membantu',
    'dengan permintaan itu',
    'feel free to ask',
    'saya tidak bisa',
]

In [29]:
count, indon_count = 0, 0

indon_texts = []
for f in tqdm(files):
    try:
        with open(f) as fopen:
            data = json.load(fopen)
    except:
        continue

    if data['output'] is None:
        continue
        
    if any([r in data['instruction'].lower() for r in rejected_instructions]):
        continue

    indon_texts.append(data['instruction'])
    indon_texts.append(data['output'])

    count += 1

100%|████████████████████████████████| 110635/110635 [00:03<00:00, 35271.74it/s]


In [30]:
len(indon_texts)

180332

In [31]:
with open('evol-instructions.texts', 'w') as fopen:
    for t in set(indon_texts):
        fopen.write(f'{json.dumps(t)}\n')

In [33]:
!cp evol-instructions.texts ~/ssd3/ctranslate2

In [34]:
mapping = {}
with open('/home/husein/ssd3/ctranslate2/evol-instructions.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
        
len(mapping)

178663

In [41]:
rejected = 0

with open('synthetic-evol-instructions.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
            
        if data['output'] is None:
            continue
        
        output_ms = mapping.get(data['output'])
        instruction_ms = mapping.get(data['instruction'])
        
        indon_ins = False
        if check_indon(data['instruction']):
            indon_ins = True
        
        indon_output = False
        if check_indon(data['output']):
            indon_output = True
        
        rejected_ins = False
        if 1024 <= len(enc.encode(data['instruction'])) <= 1025:
            rejected_ins = True
            
        if any([r in data['instruction'].lower() for r in rejected_instructions]):
            rejected_ins = True
        
        rejected_output = False
        if 1024 <= len(enc.encode(data['output'])) <= 1025:
            rejected_output = True
            
        if rejected_ins or rejected_output:
            rejected += 1
            
        data['indon_ins'] = indon_ins
        data['indon_output'] = indon_output
        data['instruction_ms'] = instruction_ms
        data['output_ms'] = output_ms
        data['rejected_ins'] = rejected_ins
        data['rejected_output'] = rejected_output
        
        fopen_l.write(f'{json.dumps(data)}\n')
        count += 1

100%|█████████████████████████████████| 110635/110635 [00:26<00:00, 4134.73it/s]


In [42]:
!wc -l synthetic-evol-instructions.jsonl

104286 synthetic-evol-instructions.jsonl
